In [1]:
%pylab inline

import lxml
from lxml import etree
import json
import os.path as op
import glob

Populating the interactive namespace from numpy and matplotlib


In [2]:
annotataion_dir  = r"e:data/scaffold/Hamamatsu/"
# annotataion_dir  = r"e:data/medical/orig/"

# Function

In [3]:
def get_one_annotation(viewstate):
    titles_list = viewstate.xpath(".//title/text()")
    if len(titles_list) == 0:
        an_title = ""
    elif len(titles_list) == 1:
        an_title = titles_list[0]
    else:
        raise ValueError("More than one title in viewstate")
    
    annotations = viewstate.xpath(".//annotation")
    if len(annotations) > 1:
        raise ValueError("More than one annotation found")
    annot = annotations[0]
    an_color = annot.get("color")
#     display(len(annotation))
    an_x = list(map(int, annot.xpath(".//pointlist/point/x/text()")))
    an_y = list(map(int, annot.xpath(".//pointlist/point/y/text()")))
    return dict(title=an_title, color=an_color, x=an_x, y=an_y)
    
    
# get_one_annotation(viewstates[2])

# Convert from .ndpa to .json

In [4]:


def ndpa_file_to_json(pth):
    tree = etree.parse(pth)
    viewstates = tree.xpath("//ndpviewstate")
    all_anotations = list(map(get_one_annotation, viewstates))
    fn = pth + ".json"
    with open(fn, 'w') as outfile:
        json.dump(all_anotations, outfile)

In [5]:
# pth = r"E:\data\scaffold\Hamamatsu\PIG-003_J-18-0165_HE.ndpi.ndpa"
# ndpa_file_to_json(pth)

# Convert whole directory

In [6]:
def ndpa_to_json(path):
    """
    :param path: path to file or dir contaning .ndpa files
    """
    if op.isfile(path):
        ndpa_file_to_json(path)
    else:
        extended_path = op.join(path, "*.ndpa")
#         print(extended_path)
        files = glob.glob(extended_path)
        for fl in files:
            ndpa_file_to_json(fl)
        
    
    

In [7]:
ndpa_to_json(annotataion_dir)